In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
def pkload(path):
    with open(path, 'rb') as f:
        obj = pickle.load(f)
    return obj

Searching for journeys from Uster, Gschwader (stop 8588052) to Buchs ZH, Furttalstrasse (stop 8595356) with arrival at 17:30 leads to a footpath of over 3.2km + in reality, while the algorithm prints this:

" Walk 2.1 minutes from Birmensdorf ZH (stop 8502221)
        to Dällikon, Industrie (stop 8576276)"

In [3]:
# Birmensdorf ZH
stop_times = pkload("../data/stop_times_df_cyril.pkl")
stop_times[stop_times['stop_id_general']==8502221].head(1)

,index,route_id,stop_id_general,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,trip_short_name,direction_id,departure_first_stop,route_int,stop_count,stop_int,route_desc,monotonically_increasing_id
27708,197013,26-5-A-j19-1,8502221,114.TA.26-5-A-j19-1.37.R,8502221:0:2,2020-05-24 19:35:00,2020-05-24 19:35:00,9,Birmensdorf ZH,47.357496,8.437543,Pfäffikon SZ,18575,1,19:22:00,149,10,276,S-Bahn,180388626589


In [55]:
# Dallikon Industrie
stop_times[stop_times['stop_id_general']==8576276].head(1)

,index,route_id,stop_id_general,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,trip_short_name,direction_id,departure_first_stop,route_int,stop_count,stop_int,route_desc,monotonically_increasing_id
1935,222352,26-449-j19-1,8576276,17.TA.26-449-j19-1.1.H,8576276,NaT,2020-05-24 07:00:00,2,"Dällikon, Industrie",47.444737,8.438783,"Buchs-Dällikon, Bahnhof",4747,0,07:00:00,19,3,474,Bus,8589935205


A google maps footpaths using GPS coordinates yields a walk of 14.1 km. Is there a footpath defined between those two stops in transfers ?

In [4]:
transfers = pkload("../data/transfer_df_cyril.pkl")

In [58]:
# transfers from Birmensdorf ZH
transfers[transfers['stop_id_general']==8502221]

,index,stop_id_general,stop_int,stop_lat_first,stop_lon_first,stop_name_first,stop_id_general_2,stop_int_2,stop_lat_first_2,stop_lon_first_2,stop_name_first_2,distance,walking_time,monotonically_increasing_id
1300,3887,8502221,276,47.357557,8.437543,Birmensdorf ZH,8573718,473,47.357125,8.438801,"Birmensdorf ZH, Bahnhof",0.106219,127,352187318287
1301,3888,8502221,276,47.357557,8.437543,Birmensdorf ZH,8502950,877,47.353936,8.437175,"Birmensdorf ZH, Zentrum",0.403584,484,352187318288
1302,3889,8502221,276,47.357557,8.437543,Birmensdorf ZH,8583870,1154,47.357234,8.437013,"Birmensdorf ZH, Bahnhof Süd",0.053666,64,352187318289


In [60]:
# Transfers from Dallikon Industrie

In [59]:
transfers[transfers['stop_id_general']==8576276]

,index,stop_id_general,stop_int,stop_lat_first,stop_lon_first,stop_name_first,stop_id_general_2,stop_int_2,stop_lat_first_2,stop_lon_first_2,stop_name_first_2,distance,walking_time,monotonically_increasing_id


There are no footpaths between Birmensdorf ZH and Dallikon Industrie, and interestingly, Dallikon Industrie has no footpaths at all. That points to a bug with the gestion of 'None' pointers in the array stops for footpaths

In [5]:
transfer_array = pkload("../data/transfer_array_cyril.pkl")
stops =  pkload("../data/stops_array_cyril.pkl")

In [64]:
# transfers from Birmensdorf ZH
stop_int = 276
stops[stop_int-1:stop_int+2]

array([[3075, 1297.0],
       [3119, 1300.0],
       [3131, 1303.0]], dtype=object)

In [65]:
transfer_array[int(stops[stop_int][1]):int(stops[stop_int+1][1])]

array([[ 473,  127],
       [ 877,  484],
       [1154,   64]])

**The first entry of transfer_array[276] corresponding to Birmensdorf ZH is 473, which is one less than 474, the stop_id of Dallikon Industrie**. That may be a clue to what's going wrong with the algorithm.

In [66]:
# transfers from Dallikon Industrie
stop_int = 474
stops[stop_int-1:stop_int+2]


array([[5408, 2157.0],
       [5417, nan],
       [5420, 2160.0]], dtype=object)

In [69]:
# Trying to access the transfer for the nan pointer (SHOULD FAIL)
transfer_array[int(stops[stop_int][1]):int(stops[stop_int+1][1])]

ValueError: cannot convert float NaN to integer

In [70]:
# checking out what's around the nan pointer:

In [71]:
transfer_array[int(stops[stop_int-1][1]):int(stops[stop_int+1][1])]

array([[ 276,  127],
       [ 877,  450],
       [1154,  162]])

The first entry of transfer_array[stop_int-1] is 276, the stop_int of Birmensdorf ZH.

## recapitulating the bug with another journey

Searching for journeys from Dübendorf, Branzenäsch (stop 8590551) to Kloten, Weinbergstrasse (stop 8579967) with arrival at 17:30 gives an impossible first walk from Dübendorf, Branzenäsch to Uetliberg (stop 8503057).

In [72]:
# Dübendorf, Branzenäsch
stop_times[stop_times['stop_id_general']== 8590551].head(1)

,index,route_id,stop_id_general,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,trip_short_name,direction_id,departure_first_stop,route_int,stop_count,stop_int,route_desc,monotonically_increasing_id
95873,110350,26-752-j19-1,8590551,190.TA.26-752-j19-1.4.R,8590551,2020-05-24 07:16:00,2020-05-24 07:16:00,11,"Dübendorf, Branzenäsch",47.394665,8.631157,"Dübendorf, Kunsteisbahn",1420,1,07:06:00,563,15,503,Bus,627065226053


In [73]:
#  Uetliberg (stop 8503057)
stop_times[stop_times['stop_id_general']== 8503057].head(1)

,index,route_id,stop_id_general,trip_id,stop_id,arrival_time,departure_time,stop_sequence,stop_name,stop_lat,stop_lon,trip_headsign,trip_short_name,direction_id,departure_first_stop,route_int,stop_count,stop_int,route_desc,monotonically_increasing_id
5304,51749,26-10-B-j19-1,8503057,181.TA.26-10-B-j19-1.9.H,8503057,2020-05-24 07:14:00,NaT,8,Uetliberg,47.352366,8.487651,Uetliberg,23511,0,07:03:00,62,4,415,S-Bahn,25769805095


A google maps search for footpaths between the coordinates yields a 14.5 km walk.

In [74]:
# footpaths from Dübendorf, Branzenäsch
transfers[transfers['stop_id_general']== 8590551]

,index,stop_id_general,stop_int,stop_lat_first,stop_lon_first,stop_name_first,stop_id_general_2,stop_int_2,stop_lat_first_2,stop_lon_first_2,stop_name_first_2,distance,walking_time,monotonically_increasing_id
2256,2938,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590587,414,47.393053,8.633367,"Dübendorf, Sonnenberg",0.244501,293,618475290624
2257,2939,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590555,599,47.394877,8.628714,"Dübendorf, Claridenstrasse",0.185439,222,618475290625
2258,2940,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590581,886,47.391557,8.634876,"Dübendorf, Raubbühl",0.444715,533,618475290626
2259,2941,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590550,931,47.397048,8.625614,"Dübendorf, Bettlistrasse",0.494279,593,618475290627
2260,2942,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590590,1226,47.390535,8.632469,"Dübendorf, Sunnhalde",0.469644,563,618475290628
2261,2943,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590562,1244,47.396240,8.629935,"Dübendorf, Glärnischstrasse",0.197806,237,618475290629
2262,2944,8590551,503,47.394665,8.631157,"Dübendorf, Branzenäsch",8590549,1392,47.394817,8.634580,"Dübendorf, Bauhof",0.258181,309,618475290630


No sign of Uetliberg here.

In [75]:
# transfers from Uetliberg (stop 8503057)
transfers[transfers['stop_id_general']== 8503057]

,index,stop_id_general,stop_int,stop_lat_first,stop_lon_first,stop_name_first,stop_id_general_2,stop_int_2,stop_lat_first_2,stop_lon_first_2,stop_name_first_2,distance,walking_time,monotonically_increasing_id


Again, utliberg has no footpaths, like Dallikon Industrie in the previous bug.

In [79]:
# footpaths from Dübendorf, Branzenäsch
stop_int = 503
stops[stop_int-1:stop_int+2]

array([[5599, 2253.0],
       [5603, 2256.0],
       [5606, 2263.0]], dtype=object)

In [80]:
transfer_array[int(stops[stop_int][1]):int(stops[stop_int+1][1])]

array([[ 414,  293],
       [ 599,  222],
       [ 886,  533],
       [ 931,  593],
       [1226,  563],
       [1244,  237],
       [1392,  309]])

Notice here again that the first transfer from Dübendorf, Branzenäsch goes to stop_int= 414, which is one less than the stop_int of Utliberg (415)

In [81]:
# footpahts from Utliberg:
stop_int = 415
stops[stop_int-1:stop_int+2]

array([[4850, 1901.0],
       [4853, nan],
       [4860, 1907.0]], dtype=object)

In [83]:
# expected to fail: accessing a none pointer
transfer_array[int(stops[stop_int][1]):int(stops[stop_int+1][1])]

ValueError: cannot convert float NaN to integer

In [84]:
transfer_array[int(stops[stop_int][1]):int(stops[stop_int+1][1])]

ValueError: cannot convert float NaN to integer

In [85]:
transfer_array[int(stops[stop_int-1][1]):int(stops[stop_int+1][1])]

array([[ 503,  293],
       [ 599,  485],
       [ 886,  241],
       [1226,  345],
       [1244,  526],
       [1392,  259]])

Again, the first entry of transfer_array[stop_int-1] is 503, the stop_int of Dübendorf, Branzenäsch !

## Testing Felix's stop conversion:

In [6]:
np.isnan(np.arange(1))

array([False])

In [7]:
stops = pkload("../data/stops_array_cyril.pkl")
stopRoutes = pkload("../data/stop_routes_array_cyril.pkl")
print(np.isnan(stops.astype(np.float64)).sum(axis=0))
print(np.equal(stops, None).sum(axis=0))
print(stops.shape)
stops = stops[:,[0,0,1,1]]
# Make column 1 contain the start_index of the next stop in stopRoutes
stops[:-1,1] = stops[1:,0]
stops[-1, 1] = stopRoutes.shape[0]
# Make column 3 contain the start_index of the next stop in stopRoutes
if np.isnan(stops[-1,2]).item():
    stops[-1,2] = transfers.shape[0]
for i in np.isnan(stops[:-1,2].astype(np.float64)).nonzero()[0][::-1]:
    stops[i,2] = stops[i+1,2]
print(np.isnan(stops.astype(np.float64)).sum(axis=0))
stops[:-1,3] = stops[1:,2]
stops[-1, 3] = transfers.shape[0]
# Convert to int
stops = stops.astype(np.uint32)
stops

[ 0 75]
[0 0]
(1407, 2)
[ 0  0  0 75]


array([[    0,    11,     0,     2],
       [   11,    20,     2,     7],
       [   20,    38,     7,    22],
       ...,
       [15303, 15334,  6242,  6250],
       [15334, 15339,  6250,  6257],
       [15339, 15344,  6257,  6264]], dtype=uint32)

In [8]:
# footpaths from Dübendorf, Branzenäsch
stop_int = 503
stops[stop_int-1:stop_int+2]

array([[5599, 5603, 2253, 2256],
       [5603, 5606, 2256, 2263],
       [5606, 5615, 2263, 2266]], dtype=uint32)

In [9]:
transfer_array[stops[stop_int][2]:stops[stop_int][3]]

array([[ 414,  293],
       [ 599,  222],
       [ 886,  533],
       [ 931,  593],
       [1226,  563],
       [1244,  237],
       [1392,  309]])

In [10]:
# footpaths from utliberg
# footpahts from Utliberg:
stop_int = 415
stops[stop_int-1:stop_int+2]


array([[4850, 4853, 1901, 1907],
       [4853, 4860, 1907, 1907],
       [4860, 4885, 1907, 1914]], dtype=uint32)

There's the bug ! The pointers for transfer_array in the first entry shows 1901, 1901 when it should in fact be the second entry that shows 1901, 1901. The first entry should show (1901, 1907) (see cell nr 81).